In [41]:
import pandas as pd

In [79]:
# Load testing_all_log.csv
df = pd.read_csv("testing_all_log.csv")
print(len(df))

# other_df = pd.read_csv("log.csv")
# first_sentence = other_df["original_text"][0].replace("[[","").replace("]]","").split(" ")

# Initialize text_key column with zeros
df['text_key'] = 0

# Iterate through rows and update text_key values
current_text_key = 0
for i in range(1, len(df)):
    if df.loc[i, 'num_queries'] < df.loc[i - 1, 'num_queries']:
        current_text_key += 1
    df.loc[i, 'text_key'] = current_text_key

290134


In [80]:
df.head()

,attacked_text,attack_model_output,num_queries,improvement,text_key
0,I love sci-fi and am willing to put up with a ...,tensor(0.0012),1,True,0
1,I love sci-fi and am willing to put up with a ...,tensor(0.0016),4,True,0
2,I love sci-fi and am willing to put up with a ...,tensor(0.0012),4,False,0
3,I love sci-fi and am willing to put up with a ...,tensor(0.0014),4,False,0
4,I love sci-fi and am willing to put up with a ...,tensor(0.0022),10,True,0


In [82]:
df["attack_model_output"][0]

'tensor(0.0012)'

In [109]:
thresholds = [0.5, 0.8, 0.97, 0.997]
successes = [0 for _ in range(len(thresholds))]
threshold_queries = [0 for _ in range(len(thresholds))]
threshold_diffs = [0 for _ in range(len(thresholds))]
skips = 0

for key in range(max(df["text_key"]) + 1):
    if key % 100 == 0:
        print(key)
    curr_row_successes = [-1 for _ in range(len(thresholds))]
    curr_row_thresholds = [0 for _ in range(len(thresholds))]
    curr_row_diffs = [0 for _ in range(len(thresholds))]
    for i, row in enumerate(df[df["text_key"] == key].iterrows()):
        row = row[1]
        if i == 0:
            base_sentence = row["attacked_text"].replace("[[","").replace("]]","").split(" ")
        if (row["num_queries"] == 1) and float(row["attack_model_output"][7:-1]) > 0.5:
            skips += 1
            continue
        # print(row)
        if float(row["attack_model_output"][7:-1]) > 0.5:
            if curr_row_successes[0] == -1:
                curr_row_successes[0] = 1
                curr_row_thresholds[0] = float(row["num_queries"])
                num_diff = sum([1 if base_sentence[i] != row["attacked_text"].replace("[[","").replace("]]","").split(" ")[i] else 0 for i in range(len(base_sentence))])
                # print(base_sentence)
                # print(row["attacked_text"].replace("[[","").replace("]]","").split(" "))
                
                # print([1 if base_sentence[i] != row["attacked_text"].replace("[[","").replace("]]","").split(" ")[i] else 0 for i in range(len(base_sentence))])
                # print(num_diff)
                # err
                curr_row_diffs[0] = num_diff
        if float(row["attack_model_output"][7:-1]) > 0.8:
            if curr_row_successes[1] == -1:
                curr_row_successes[1] = 1
                curr_row_thresholds[1] = float(row["num_queries"])
                num_diff = sum([1 if base_sentence[i] != row["attacked_text"].replace("[[","").replace("]]","").split(" ")[i] else 0 for i in range(len(base_sentence))])
                curr_row_diffs[1] = num_diff
        if float(row["attack_model_output"][7:-1]) > 0.97:
            if curr_row_successes[2] == -1:
                curr_row_successes[2] = 1
                curr_row_thresholds[2] = float(row["num_queries"])
                num_diff = sum([1 if base_sentence[i] != row["attacked_text"].replace("[[","").replace("]]","").split(" ")[i] else 0 for i in range(len(base_sentence))])
                curr_row_diffs[2] = num_diff
        if float(row["attack_model_output"][7:-1]) > 0.997:
            if curr_row_successes[3] == -1:
                curr_row_successes[3] = 1
                curr_row_thresholds[3] = float(row["num_queries"])
                num_diff = sum([1 if base_sentence[i] != row["attacked_text"].replace("[[","").replace("]]","").split(" ")[i] else 0 for i in range(len(base_sentence))])
                curr_row_diffs[3] = num_diff
            
    for i in range(len(thresholds)):
        if curr_row_successes[i] == 1:
            successes[i] += 1
            threshold_queries[i] += curr_row_thresholds[i]
            threshold_diffs[i] += curr_row_diffs[i] / len(base_sentence)


    

0
100
200
300
400
500


In [110]:
l1 = ['I', 'love', 'sci-fi', 'and', 'am', 'willing', 'to', 'put', 'up', 'with', 'a', 'lot.', 'Sci-fi', 'movies/TV', 'are', 'usually', 'underfunded,', 'under-appreciated', 'and', 'misunderstood.', 'I', 'tried', 'to', 'like', 'this,', 'I', 'really', 'did,', 'but', 'it', 'is', 'to', 'good', 'TV', 'sci-fi', 'as', 'Babylon', '5', 'is', 'to', 'Star', 'Trek', '(the', 'original).', 'Silly', 'prosthetics,', 'cheap', 'cardboard', 'sets,', 'stilted', 'dialogues,', 'CG', 'that', "doesn't", 'match', 'the', 'background,', 'and', 'painfully', 'one-dimensional', 'characters', 'cannot', 'be', 'overcome', 'with', 'a', "'sci-fi'", 'setting.', "(I'm", 'sure', 'there', 'are', 'those', 'of', 'you', 'out', 'there', 'who', 'think', 'Babylon', '5', 'is', 'good', 'sci-fi', 'TV.', "It's", 'not.', "It's", 'clichéd', 'and', 'uninspiring.)', 'While', 'US', 'viewers', 'might', 'like', 'emotion', 'and', 'character', 'development,', 'sci-fi', 'is', 'a', 'genre', 'that', 'does', 'not', 'take', 'itself', 'seriously', '(cf.', 'Star', 'Trek).', 'It', 'may', 'treat', 'important', 'issues,', 'yet', 'not', 'as', 'a', 'serious', 'philosophy.', "It's", 'really', 'difficult', 'to', 'care', 'about', 'the', 'characters', 'here', 'as', 'they', 'are', 'not', 'simply', 'foolish,', 'just', 'missing', 'a', 'spark', 'of', 'life.', 'Their', 'actions', 'and', 'reactions', 'are', 'wooden', 'and', 'predictable,', 'often', 'painful', 'to', 'watch.', 'The', 'makers', 'of', 'Earth', 'KNOW', "it's", 'rubbish', 'as', 'they', 'have', 'to', 'always', 'say', '"Gene', "Roddenberry's", 'Earth..."', 'otherwise', 'people', 'would', 'not', 'continue', 'watching.', "Roddenberry's", 'ashes', 'must', 'be', 'turning', 'in', 'their', 'orbit', 'as', 'this', 'dull,', 'cheap,', 'poorly', 'edited', '(watching', 'it', 'without', 'advert', 'breaks', 'really', 'brings', 'this', 'home)', 'trudging', 'Trabant', 'of', 'a', 'show', 'lumbers', 'into', 'space.', 'Spoiler.', 'So,', 'kill', 'off', 'a', 'main', 'character.', 'And', 'then', 'bring', 'him', 'back', 'as', 'another', 'actor.', 'Jeeez!', 'Dallas', 'all', 'over', 'again.']
l2 = ['I', 'love', 'sci-fi', 'and', 'am', 'willing', 'to', 'put', 'up', 'with', 'a', 'lot.', 'Sci-fi', 'cinematography/TV', 'are', 'often', 'underfunded,', 'under-appreciated', 'and', 'misunderstood.', 'I', 'attempt', 'to', 'like', 'this,', 'I', 'really', 'did,', 'but', 'it', 'is', 'to', 'well', 'TV', 'sci-fi', 'as', 'Babylon', '5', 'is', 'to', 'Star', 'Trek', '(the', 'original).', 'Dumb', 'prosthetics,', 'cheap', 'cardboard', 'sets,', 'stilted', 'talks,', 'CG', 'that', "doesn't", 'match', 'the', 'background,', 'and', 'painfully', 'one-dimensional', 'characters', 'cannot', 'be', 'overcome', 'with', 'a', "'sci-fi'", 'setting.', "(I'm", 'sure', 'there', 'are', 'those', 'of', 'you', 'out', 'there', 'who', 'think', 'Babylon', '5', 'is', 'good', 'sci-fi', 'TV.', "It's", 'not.', "It's", 'clichéd', 'and', 'uninspiring.)', 'While', 'US', 'viewers', 'might', 'like', 'emotion', 'and', 'character', 'development,', 'sci-fi', 'is', 'a', 'genre', 'that', 'does', 'not', 'take', 'itself', 'seriously', '(cf.', 'Star', 'Hiking).', 'It', 'may', 'treat', 'important', 'issues,', 'yet', 'not', 'as', 'a', 'serious', 'philosophy.', "It's", 'really', 'difficult', 'to', 'care', 'about', 'the', 'characters', 'here', 'as', 'they', 'are', 'not', 'simply', 'foolish,', 'just', 'missing', 'a', 'spark', 'of', 'life.', 'Their', 'measures', 'and', 'reaction', 'are', 'wood', 'and', 'predictable,', 'often', 'painful', 'to', 'watch.', 'The', 'manufacturers', 'of', 'Earth', 'KNOW', "it's", 'litter', 'as', 'they', 'have', 'to', 'always', 'says', '"Genetics', "Roddenberry's", 'Earth..."', 'else', 'people', 'would', 'not', 'continue', 'watching.', "Roddenberry's", 'cinder', 'gotta', 'be', 'turning', 'in', 'their', 'orbit', 'as', 'this', 'drab,', 'cheaper,', 'poorly', 'edited', '(staring', 'it', 'without', 'ads', 'breaks', 'really', 'brings', 'this', 'home)', 'trudging', 'Trabant', 'of', 'a', 'exhibitions', 'lumbers', 'into', 'space.', 'Baffle.', 'Therefore,', 'kiiled', 'off', 'a', 'main', 'nature.', 'And', 'then', 'bring', 'him', 'back', 'as', 'another', 'protagonist.', 'Jeeez!', 'Dallas', 'all', 'over', 'again.']

for x,y in zip(l1,l2):
    if x != y:
        print(x,y)

movies/TV cinematography/TV
usually often
tried attempt
good well
Silly Dumb
dialogues, talks,
Trek). Hiking).
actions measures
reactions reaction
wooden wood
makers manufacturers
rubbish litter
say says
"Gene "Genetics
otherwise else
ashes cinder
must gotta
dull, drab,
cheap, cheaper,
(watching (staring
advert ads
show exhibitions
Spoiler. Baffle.
So, Therefore,
kill kiiled
character. nature.
actor. protagonist.


In [111]:
print("Skips: {}".format(skips))

for i in range(len(thresholds)):
    success_rate = successes[i] / (1 + max(df["text_key"]) - skips)
    avg_queries = threshold_queries[i] / successes[i]
    avg_diffs = threshold_diffs[i] / successes[i]
    
    print("Success rate for threshold {}: {}".format(thresholds[i], success_rate))
    print("Average queries for threshold {}: {}".format(thresholds[i], avg_queries))
    print("Average diffs for threshold {}: {}".format(thresholds[i], avg_diffs))
    print()
    

Skips: 29
Success rate for threshold 0.5: 0.9722735674676525
Average queries for threshold 0.5: 111.25285171102662
Average diffs for threshold 0.5: 0.07223578754254072

Success rate for threshold 0.8: 0.9426987060998152
Average queries for threshold 0.8: 131.2764705882353
Average diffs for threshold 0.8: 0.0829187879834899

Success rate for threshold 0.97: 0.866913123844732
Average queries for threshold 0.97: 170.62473347547976
Average diffs for threshold 0.97: 0.10171808600078702

Success rate for threshold 0.997: 0.6229205175600739
Average queries for threshold 0.997: 269.85459940652817
Average diffs for threshold 0.997: 0.14608716049900713



In [ ]:
Skips: 29
Success rate for threshold 0.5: 0.9722735674676525
Average queries for threshold 0.5: 523.8517110266159

Success rate for threshold 0.8: 0.9426987060998152
Average queries for threshold 0.8: 527.8764705882353

Success rate for threshold 0.97: 0.866913123844732
Average queries for threshold 0.97: 540.8379530916844

Success rate for threshold 0.997: 0.6229205175600739
Average queries for threshold 0.997: 549.6112759643917

In [91]:
1 + max(df["text_key"])

570

In [76]:
# Get number of unique original texts
print(len(df['attacked_text'].unique()))

290134


In [77]:
improvements = df[df["improvement"]]

improvements.head()

,attacked_text,attack_model_output,num_queries,improvement
0,I love sci-fi and am willing to put up with a ...,tensor(0.0012),1,True
1,I love sci-fi and am willing to put up with a ...,tensor(0.0016),4,True
4,I love sci-fi and am willing to put up with a ...,tensor(0.0022),10,True
5,I love sci-fi and am willing to put up with a ...,tensor(0.0026),10,True
12,I love sci-fi and am willing to put up with a ...,tensor(0.0033),15,True


In [78]:
def print_text(text):
    for i in range(0, len(text), 100):
        print(text[i:i+100])

In [64]:
improvements[improvements["text_key"] == 1]["attack_model_output"].values
print((improvements[improvements["text_key"] == 1]["attack_model_output"].values))

['tensor(0.0072)' 'tensor(0.0100)' 'tensor(0.0136)' 'tensor(0.0137)'
 'tensor(0.0166)' 'tensor(0.0336)' 'tensor(0.0412)' 'tensor(0.0475)'
 'tensor(0.0683)' 'tensor(0.1559)' 'tensor(0.1730)' 'tensor(0.1754)'
 'tensor(0.1854)' 'tensor(0.1854)' 'tensor(0.1888)' 'tensor(0.5429)'
 'tensor(0.5961)' 'tensor(0.5970)' 'tensor(0.6439)' 'tensor(0.6622)'
 'tensor(0.6622)' 'tensor(0.7835)' 'tensor(0.8035)' 'tensor(0.8328)'
 'tensor(0.8865)' 'tensor(0.9312)' 'tensor(0.9378)' 'tensor(0.9389)'
 'tensor(0.9562)' 'tensor(0.9578)' 'tensor(0.9669)' 'tensor(0.9799)'
 'tensor(0.9862)' 'tensor(0.9885)' 'tensor(0.9897)' 'tensor(0.9904)'
 'tensor(0.9915)' 'tensor(0.9922)' 'tensor(0.9926)' 'tensor(0.9936)'
 'tensor(0.9936)' 'tensor(0.9939)' 'tensor(0.9940)' 'tensor(0.9941)'
 'tensor(0.9947)' 'tensor(0.9954)' 'tensor(0.9954)' 'tensor(0.9955)'
 'tensor(0.9956)' 'tensor(0.9965)' 'tensor(0.9966)' 'tensor(0.9968)'
 'tensor(0.9970)' 'tensor(0.9970)' 'tensor(0.9971)' 'tensor(0.9972)'
 'tensor(0.9973)' 'tensor(0.9973)'

In [46]:
# Add a column for each input_text. Determine this because any time num_queries is 1, it's a new input_text.
df['text_key'] = 0
